In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import networkx as nx
import os
from sklearn.metrics import mutual_info_score
from scipy.stats import zscore
import pandas as pd
from sklearn.preprocessing import LabelEncoder
#import packages
import pandas as pd
import wget
from datetime import date
import gzip
import pandas as pd
from itertools import combinations

In [3]:
drug_response = pd.read_csv("data/drug_response.csv", index_col=0)
drug_response = drug_response[drug_response['TCGA_DESC'] != 'UNCLASSIFIED']
# Count occurrences of each value in 'TCGA_DESC'
value_counts = drug_response['TCGA_DESC'].value_counts()
# Filter to keep only those values that appear more than 10,000 times
values_to_keep = value_counts[value_counts > 10000].index
# Filter the DataFrame to include only rows where 'TCGA_DESC' is in the values_to_keep
drug_response = drug_response[drug_response['TCGA_DESC'].isin(values_to_keep)]
models = drug_response.index.drop_duplicates()
len(models)

634

## Read and Preprocess data

### Transcriptomics data

In [4]:
gene_expression = pd.read_csv("data/gene_expression.csv", index_col=0)
gene_expression = gene_expression[gene_expression.index.isin(models)]
gene_expression

,SIDG00002,SIDG00004,SIDG00011,SIDG00017,SIDG00019,SIDG00020,SIDG00021,SIDG00026,SIDG00027,SIDG00028,...,SIDG42467,SIDG42468,SIDG42469,SIDG42471,SIDG42472,SIDG42474,SIDG42475,SIDG42479,SIDG42480,SIDG42481
SIDM00023,1.03,1.00,38.54,45.47,14.11,2.32,3.53,9.68,17.76,19.28,...,11.51,16.27,12.69,34.96,72.27,21.14,22.09,20.73,3.03,2.62
SIDM00040,1.00,1.12,3.17,33.76,24.11,1.00,1.00,1.00,2.79,12.27,...,8.40,5.71,11.34,15.36,76.67,9.09,28.97,10.71,5.36,1.40
SIDM00042,1.10,1.28,95.97,32.71,11.32,1.00,20.24,23.23,13.65,4.28,...,10.58,31.63,1.28,11.25,43.66,17.41,17.62,34.39,6.19,1.21
SIDM00043,1.04,1.00,59.07,35.16,11.31,1.33,1.04,1.22,4.88,28.87,...,30.39,28.39,1.41,31.24,93.63,15.82,30.83,6.40,5.07,1.67
SIDM00044,18.15,1.00,1.56,40.04,15.02,1.07,1.00,1.07,22.18,52.57,...,9.02,21.88,1.15,34.93,362.71,16.03,52.27,9.61,3.68,3.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SIDM01240,12.87,246.00,1.00,39.69,9.38,1.08,1.00,1.00,11.25,7.85,...,6.47,24.85,1.06,24.51,143.78,17.26,16.27,4.82,4.60,4.51
SIDM01242,16.93,28.72,1.00,44.69,9.78,1.17,1.00,1.00,10.06,44.27,...,10.75,12.23,1.17,29.65,97.33,26.09,55.40,12.34,9.30,8.29
SIDM01245,10.96,1.00,1.46,54.17,14.33,42.76,1.00,1.00,11.95,1.64,...,16.24,32.83,1.21,20.50,17.73,21.10,21.49,19.61,11.70,1.57
SIDM01246,10.06,5.23,1.47,49.80,4.69,1.16,1.00,1.06,19.40,7.75,...,12.02,38.24,1.44,14.82,147.83,7.53,35.09,6.24,6.21,1.47


#### Prepare data for inferelator

In [36]:
tfs = pd.read_csv("data/tf.txt")
tfs = list(tfs['ID'].values)

gene_ids = pd.read_csv("data/gene_identifiers_20191101.csv")[['gene_id','ensembl_gene_id','cosmic_gene_symbol']]
gene_ids.dropna(inplace=True)
gene_ids = gene_ids[gene_ids['gene_id'].isin(gene_expression.columns)].reset_index(drop=True)

In [38]:
gene_expression = gene_expression[list(gene_ids['gene_id'])]
mapping = dict(zip(gene_ids['gene_id'], gene_ids['cosmic_gene_symbol']))
gene_expression.rename(columns=mapping, inplace=True)
gene_expression

/home/surabhi/anaconda3/envs/patstrat/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,A2M,A4GALT,AAAS,AACS,AADAC,AADAT,AAGAB,AAK1,AAMP,AARS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
SIDM00023,1.00,38.54,45.47,14.11,3.53,17.76,93.28,10.02,94.53,200.37,...,75.17,144.41,4.78,11.51,16.27,12.69,34.96,72.27,21.14,22.09
SIDM00040,1.12,3.17,33.76,24.11,1.00,2.79,39.72,4.50,65.69,383.53,...,23.04,10.53,2.36,8.40,5.71,11.34,15.36,76.67,9.09,28.97
SIDM00042,1.28,95.97,32.71,11.32,20.24,13.65,43.66,16.19,98.16,166.09,...,17.48,61.99,3.82,10.58,31.63,1.28,11.25,43.66,17.41,17.62
SIDM00043,1.00,59.07,35.16,11.31,1.04,4.88,76.89,26.06,111.15,179.76,...,22.51,40.63,12.57,30.39,28.39,1.41,31.24,93.63,15.82,30.83
SIDM00044,1.00,1.56,40.04,15.02,1.00,22.18,53.36,16.18,79.09,160.90,...,43.96,106.64,3.39,9.02,21.88,1.15,34.93,362.71,16.03,52.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SIDM01240,246.00,1.00,39.69,9.38,1.00,11.25,31.50,11.11,107.70,166.07,...,28.79,80.93,2.78,6.47,24.85,1.06,24.51,143.78,17.26,16.27
SIDM01242,28.72,1.00,44.69,9.78,1.00,10.06,61.04,13.58,85.79,186.99,...,25.61,31.73,5.15,10.75,12.23,1.17,29.65,97.33,26.09,55.40
SIDM01245,1.00,1.46,54.17,14.33,1.00,11.95,69.27,16.14,113.12,432.87,...,36.06,137.61,5.11,16.24,32.83,1.21,20.50,17.73,21.10,21.49
SIDM01246,5.23,1.47,49.80,4.69,1.00,19.40,51.00,8.92,64.38,307.38,...,28.40,52.07,6.27,12.02,38.24,1.44,14.82,147.83,7.53,35.09


In [39]:
tf_sidg = gene_ids[gene_ids['ensembl_gene_id'].isin(tfs)].reset_index(drop=True)['cosmic_gene_symbol']
tf_sidg.dropna(inplace=True)

In [40]:
refnet = pd.read_csv("data/Human_TFtarget_GS.csv",header=None)
refnet = refnet[refnet[0].isin(gene_expression.columns)]
refnet = refnet[refnet[1].isin(gene_expression.columns)]
G = nx.from_pandas_edgelist(refnet, source=0, target=1)
adj_matrix = nx.to_pandas_adjacency(G, dtype=int)
refnet = adj_matrix[adj_matrix.columns.isin(tf_sidg)].T
refnet.drop_duplicates(inplace=True)
refnet

,ADNP,ADNP2,AHDC1,CIC,FOXE1,FOXG1,FOXJ2,FOXK2,FOXQ1,HNF4A,...,ZNF675,ZNF714,ZNF85,ZNF518B,ZXDA,ZBED2,INSM2,ZNF404,TBX18,ZNF90
ADNP,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
ABTB2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACTC1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ADNP2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AHDC1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FCRL3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TBX18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SAGE1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FAM83B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
cancers = drug_response['TCGA_DESC'].unique()
cancers

# Define the directory path
rnaDIR = 'data/transcriptomics/'

# Create the directory path if it doesn't exist
os.makedirs(rnaDIR, exist_ok=True)
for cancer in cancers:
    cancer_samples = list(drug_response[drug_response['TCGA_DESC'] == cancer].index)
    sub_gene_expression = gene_expression[gene_expression.index.isin(cancer_samples)]
    sub_gene_expression.to_csv(os.path.join(rnaDIR, cancer + '_gene_expression.tsv.gz'), sep='\t', index=True)

tf_sidg.to_csv(os.path.join(rnaDIR,'tf_names.tsv'), sep='\t', index=False, header=None)
refnet.to_csv(os.path.join(rnaDIR,'refnet.tsv.gz'), sep='\t', index=True)

### Proteomics Data

In [4]:
prot_expression = pd.read_csv("data/protein_abundance.csv", index_col=0)
prot_expression = prot_expression[prot_expression.index.isin(models)]
prot_expression = prot_expression.add_prefix('P_')



,P_A0FGR8,P_A1L0T0,P_A5YKK6,P_A6NHQ2,P_A6NHR9,P_E9PAV3,P_E9PRG8,P_O00116,P_O00154,P_O00165,...,P_Q9Y676,P_Q9Y678,P_Q9Y679,P_Q9Y697,P_Q9Y6C9,P_Q9Y6D9,P_Q9Y6E2,P_Q9Y6I9,P_Q9Y6M9,P_Q9Y6W5
SIDM00023,4.97764,5.25379,5.13866,6.30761,4.62630,6.01270,5.42488,5.07084,5.899690,4.79563,...,5.20974,5.29555,4.25246,5.41326,6.09241,5.35411,6.18875,4.18724,5.49440,5.86868
SIDM00040,5.53991,5.42006,5.16105,5.87183,4.30398,7.62334,5.42636,4.76346,6.943520,4.32730,...,4.08379,6.89432,4.02638,5.35204,6.65149,4.16955,6.23238,3.28483,3.77586,4.77569
SIDM00042,4.53864,4.55892,1.00000,6.63237,4.98601,1.00000,3.54964,5.94454,1.716592,4.85426,...,5.26742,2.87530,4.28659,5.58699,6.59893,3.07564,1.00000,3.91875,4.41980,3.40811
SIDM00043,5.11209,4.93324,4.28990,6.68134,4.33806,4.90732,6.37099,5.77416,5.372270,4.96512,...,4.80818,5.37741,5.44673,5.42021,6.15138,5.22480,5.54968,3.60773,4.79681,4.77634
SIDM00044,5.61874,4.53136,5.07061,6.55599,5.10544,5.65099,4.44949,5.84333,7.417010,3.94072,...,4.52047,6.02864,4.24748,4.88920,7.14342,4.93287,3.67884,4.54200,4.34279,6.19287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SIDM01240,6.28560,5.18862,4.68118,7.73827,4.23758,5.51422,5.72557,5.31587,7.237850,5.32866,...,5.56745,4.62400,4.33097,5.59300,6.77189,4.94808,6.07595,4.23484,5.87070,4.46966
SIDM01242,5.82765,5.96566,5.10414,5.62836,4.49009,7.03043,5.49964,5.01015,8.772170,4.41231,...,4.65841,6.11462,4.78253,5.70903,7.46932,4.50921,6.30197,5.49546,5.13037,5.45619
SIDM01245,6.16692,4.71188,5.62279,6.02455,4.18876,6.92108,4.15890,5.40091,7.796730,3.54500,...,3.62372,5.57740,3.37882,4.18096,6.39716,4.38834,5.97779,3.39401,3.50164,6.27499
SIDM01246,3.57599,5.26228,5.38444,7.39529,5.10903,7.47086,4.45413,5.69653,6.119120,4.72504,...,4.67935,4.94913,3.33837,4.63985,6.79813,4.40173,6.66445,3.64123,4.59901,4.52208


In [6]:
# To download string network
#String_network_url = "https://stringdb-static.org/download/protein.links.detailed.v11.5/9606.protein.links.detailed.v11.5.txt.gz"
#String_url_file = os.path.join('data',"String_PPI.txt.gz")
#response = wget.download(String_network_url, String_url_file)
#string_info_dat = pd.read_csv(String_info_url_file, sep= ' ')
string_ppi_dat = pd.read_csv(os.path.join('data',"String_PPI.txt.gz"), sep= ' ')
string_ppi_dat = string_ppi_dat[string_ppi_dat['experimental']>700].reset_index(drop=True)
string_ppi_dat = string_ppi_dat[['protein1','protein2']]
string_ppi_dat

,protein1,protein2
0,9606.ENSP00000000233,9606.ENSP00000328551
1,9606.ENSP00000000233,9606.ENSP00000262305
2,9606.ENSP00000000412,9606.ENSP00000438085
3,9606.ENSP00000001008,9606.ENSP00000360609
4,9606.ENSP00000001008,9606.ENSP00000335153
...,...,...
99533,9606.ENSP00000485663,9606.ENSP00000436049
99534,9606.ENSP00000485663,9606.ENSP00000248342
99535,9606.ENSP00000485663,9606.ENSP00000416255
99536,9606.ENSP00000485663,9606.ENSP00000220849


In [7]:
# To download string network metadata
#String_info_url = "https://stringdb-static.org/download/protein.info.v11.5/9606.protein.info.v11.5.txt.gz"
String_info_url_file = os.path.join('data',"String_protein_info.txt.gz")
#response = wget.download(String_info_url, String_info_url_file)

#string_ppi_dat
string_info_dat = pd.read_csv(String_info_url_file, sep='\t')
string_id_dict = dict(zip(string_info_dat['#string_protein_id'], string_info_dat['preferred_name']))


In [8]:
string_net = string_ppi_dat.replace(string_id_dict)

In [ ]:
# Define the prefix
prefix = 'P_'
# Add the prefix to all elements in the DataFrame
string_net = string_net.applymap(lambda x: prefix + x)


      protein1     protein2
0       P_ARF5      P_RAB28
1       P_ARF5  P_RAB11FIP3
2       P_M6PR       P_GGA3
3      P_FKBP4   P_HSP90AB1
4      P_FKBP4   P_HSP90AA1
...        ...          ...
99533  P_EIF3L      P_EIF3M
99534  P_EIF3L      P_EIF3K
99535  P_EIF3L     P_EIF4G1
99536  P_EIF3L      P_EIF3E
99537  P_EIF3L       P_RPS2

[99538 rows x 2 columns]


In [ ]:
cancers = drug_response['TCGA_DESC'].unique()
cancers

# Define the directory path
protDIR = 'results/proteomics/'

# Create the directory path if it doesn't exist
os.makedirs(protDIR, exist_ok=True)
for cancer in cancers:
    cancer_samples = list(drug_response[drug_response['TCGA_DESC'] == cancer].index)
    sub_prot_expression = prot_expression[prot_expression.index.isin(cancer_samples)]
    string_net1 = string_net[string_net['protein1'].isin(sub_prot_expression.columns)]
    string_net2 = string_net1[string_net1['protein2'].isin(sub_prot_expression.columns)]
    mutual_info_matrix = compute_mutual_information(sub_prot_expression)
    protnet = pd.concat(mutual_info_matrix,string_net2)
    print(protnet)
    protnet.to_csv(os.path.join(protDIR, cancer + '_prot_net.tsv.gz'), sep='\t', index=True)


### Mutation data

In [5]:
mut = pd.read_csv('data/mutations_all_20230202.csv')
mut = mut[mut['model_id'].isin(models)].reset_index(drop=True)
mut.index = mut['model_id'].values
mut

,gene_id,gene_symbol,model_id,protein_mutation,rna_mutation,cdna_mutation,cancer_driver,cancer_predisposition_variant,effect,vaf,coding,source,model_name
SIDM00347,SIDG03759,CCDC171,SIDM00347,-,r.?,-,False,False,intronic,1.0000,False,Sanger,TE-5
SIDM00347,SIDG05041,CNTLN,SIDM00347,-,r.?,-,False,False,intronic,0.5357,False,Sanger,TE-5
SIDM00347,SIDG05041,CNTLN,SIDM00347,p.?,r.4302g>u,c.?,False,False,3prime_UTR_variant,1.0000,False,Sanger,TE-5
SIDM00347,SIDG00421,ADAMTSL1,SIDM00347,-,r.?,-,False,False,intronic,1.0000,False,Sanger,TE-5
SIDM00347,SIDG00421,ADAMTSL1,SIDM00347,-,r.?,-,False,False,intronic,0.9167,False,Sanger,TE-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
SIDM00663,SIDG00920,AMER1,SIDM00663,p.E227K,r.916g>a,c.679G>A,False,False,missense,0.4100,True,Broad,Hs-939-T
SIDM00663,SIDG24038,PCDH11X,SIDM00663,-,r.?,-,False,False,intronic,0.3793,False,Broad,Hs-939-T
SIDM00663,SIDG37453,TENM1,SIDM00663,p.P1242S,r.3788c>u,c.3724C>T,False,False,missense,0.5309,True,Broad,Hs-939-T
SIDM00663,SIDG37453,TENM1,SIDM00663,-,r.?,-,False,False,intronic,0.5422,False,Broad,Hs-939-T


In [14]:
prot_mut = mut[['gene_symbol', 'protein_mutation']]
rna_mut = mut[['gene_symbol', 'rna_mutation']]
cdna_mut = mut[['gene_symbol', 'cdna_mutation']]

In [18]:
# Assuming drug_response, prot_mut, cdna_mut, and rna_mut are already defined
cancers = drug_response['TCGA_DESC'].unique()

# Define the directory path
mutDIR = 'results/genomics/'

# Create the directory path if it doesn't exist
os.makedirs(mutDIR, exist_ok=True)

for cancer in cancers:
    all_mut = pd.DataFrame(columns=['gene_symbol', 'mutation'])
    for mutation_table in [prot_mut, cdna_mut, rna_mut]:
        mutation_table = mutation_table[(mutation_table.iloc[:, 1] != '-') & 
                                        (mutation_table.iloc[:, 1] != 'p.?') & 
                                        (mutation_table.iloc[:, 1] != 'r.?') &
                                        (mutation_table.iloc[:, 1] != 'c.?')].reset_index(drop=True)
        mutation_table.columns = ['gene_symbol', 'mutation']
        all_mut = pd.concat([all_mut, mutation_table], ignore_index=True)
    
    # Save to CSV
    output_file = os.path.join(mutDIR, f'{cancer}_mutations.tsv.gz')
    all_mut.to_csv(output_file, sep='\t', index=False)

### PSN- Genomics

,gene_id,gene_symbol,model_id,protein_mutation,rna_mutation,cdna_mutation,cancer_driver,cancer_predisposition_variant,effect,vaf,coding,source,model_name
SIDM00347,SIDG03759,CCDC171,SIDM00347,-,r.?,-,False,False,intronic,1.0000,False,Sanger,TE-5
SIDM00347,SIDG05041,CNTLN,SIDM00347,-,r.?,-,False,False,intronic,0.5357,False,Sanger,TE-5
SIDM00347,SIDG05041,CNTLN,SIDM00347,p.?,r.4302g>u,c.?,False,False,3prime_UTR_variant,1.0000,False,Sanger,TE-5
SIDM00347,SIDG00421,ADAMTSL1,SIDM00347,-,r.?,-,False,False,intronic,1.0000,False,Sanger,TE-5
SIDM00347,SIDG00421,ADAMTSL1,SIDM00347,-,r.?,-,False,False,intronic,0.9167,False,Sanger,TE-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
SIDM00663,SIDG00920,AMER1,SIDM00663,p.E227K,r.916g>a,c.679G>A,False,False,missense,0.4100,True,Broad,Hs-939-T
SIDM00663,SIDG24038,PCDH11X,SIDM00663,-,r.?,-,False,False,intronic,0.3793,False,Broad,Hs-939-T
SIDM00663,SIDG37453,TENM1,SIDM00663,p.P1242S,r.3788c>u,c.3724C>T,False,False,missense,0.5309,True,Broad,Hs-939-T
SIDM00663,SIDG37453,TENM1,SIDM00663,-,r.?,-,False,False,intronic,0.5422,False,Broad,Hs-939-T


,gene_symbol
SIDM01016,A1BG
SIDM00315,A1BG
SIDM00339,A1BG
SIDM00562,A1BG
SIDM00727,A1BG
...,...
SIDM00358,ZZEF1
SIDM00358,ZZEF1
SIDM01092,ZZEF1
SIDM00661,ZZEF1


,patient1,patient2
0,SIDM00150,SIDM00886
1,SIDM00193,SIDM00886
2,SIDM00688,SIDM00886
3,SIDM00433,SIDM00886
4,SIDM00886,SIDM01007
...,...,...
201290,SIDM00921,SIDM01083
201291,SIDM00342,SIDM00513
201292,SIDM00212,SIDM00404
201293,SIDM00125,SIDM00125


In [13]:
pairs_df = pairs_df.apply(lambda row: pd.Series(sorted(row)), axis=1)
pairs_df.columns = ['patient1', 'patient2']

In [16]:
pairs_df

,patient1,patient2
0,SIDM00150,SIDM00886
1,SIDM00193,SIDM00886
2,SIDM00688,SIDM00886
3,SIDM00433,SIDM00886
4,SIDM00886,SIDM01007
...,...,...
201290,SIDM00921,SIDM01083
201291,SIDM00342,SIDM00513
201292,SIDM00212,SIDM00404
201293,SIDM00125,SIDM00125


In [10]:



# Group by gene symbol and create pairwise combinations of indices
pairs_list = []
for gene, group in data.groupby('gene_symbol'):
    indices = group.index.tolist()
    pairs = list(combinations(indices, 2))  # Generate pairwise combinations
    pairs_list.extend(pairs)

# Create a DataFrame for the pairs
pairs_df = pd.DataFrame(pairs_list, columns=['patient1', 'patient2'])

# Ensure pairs are in the same order
pairs_df = pairs_df.apply(lambda row: pd.Series(sorted(row)), axis=1)
pairs_df.columns = ['patient1', 'patient2']

# Count occurrences of each pair
pairs_count_df = pairs_df.groupby(['patient1', 'patient2']).size().reset_index(name='similarity_score')

print(pairs_count_df)

: 

In [1]:
pairs_df

NameError: name 'pairs_df' is not defined